In [1]:
import pandas as pd
from ast import literal_eval
meta = pd.read_csv("movies_metadata.csv", na_filter=True)
cast = pd.read_csv("credits.csv", na_filter=True)
pd.set_option("display.max_columns", None)

cast['cast'] = cast['cast'].apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
cast['crew'] = cast['crew'].apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
meta['genres'] = meta['genres'].apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

meta['belongs_to_collection'] = meta['belongs_to_collection'].fillna('{}')
meta['belongs_to_collection'].apply(literal_eval)

meta['id'] = pd.to_numeric(meta['id'], errors='coerce')
meta['popularity'] = pd.to_numeric(meta['popularity'], errors='coerce')

data = pd.merge(meta, cast, on='id')
data = data.dropna(subset=['popularity'])
data = data[data['popularity'] > 10]

data.reset_index(drop=True, inplace=True)

len(data)

C:\Users\alexp\AppData\Local\Temp\ipykernel_53508\3902482140.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  meta = pd.read_csv("movies_metadata.csv", na_filter=True)


3023

In [2]:
# set of genres

genres = set()
for row in meta['genres']:
    for genre in row:
        genres.add(genre)

collections = set()
for row in meta['belongs_to_collection']:
    try:
        collections.add(literal_eval(row)['name'])
    except:
        pass

In [3]:
import numpy as np

nodes = []

# add movie nodes
for index, row in data.iterrows():
    if pd.notnull(row['overview']) and row['overview'] != '' and pd.notnull(row['title']):
        node = {}
        node['id'] = row['id']
        node['title'] = row['title']
        node['overview'] = row['overview']
        node['genres'] = row['genres']
        node['cast'] = row['cast']
        node['belongs_to_collection'] = row['belongs_to_collection']
        nodes.append(node)

# add collection nodes

# add actor nodes
#for index, row in data.iterrows():
#    for actor in row['cast'][:5]:
#        if pd.notnull(actor) and actor != '':
#            node = {}
#            node['id'] = actor
#            node['title'] = actor
#            node['overview'] = ''
#            node['genres'] = []
#            node['cast'] = []
#            nodes.append(node)

# add genre nodes
for i, genre in enumerate(genres):
    nodes.append({"id": i, "title": genre, "overview": "", "genres": [], "cast": [], "belongs_to_collection": []})

# add collection nodes
for i, collection in enumerate(collections):
    nodes.append({"id": collection, "title": collection, "overview": "", "genres": [], "cast": [], "belongs_to_collection": []})
len(nodes)

4750

In [4]:
nodes

[{'id': 862.0,
  'title': 'Toy Story',
  'overview': "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",
  'genres': ['Animation', 'Comedy', 'Family'],
  'cast': ['Tom Hanks',
   'Tim Allen',
   'Don Rickles',
   'Jim Varney',
   'Wallace Shawn',
   'John Ratzenberger',
   'Annie Potts',
   'John Morris',
   'Erik von Detten',
   'Laurie Metcalf',
   'R. Lee Ermey',
   'Sarah Freeman',
   'Penn Jillette'],
  'belongs_to_collection': "{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"},
 {'id': 8844.0,
  'title': 'Jumanji',
  'overview': "When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world,

In [5]:
nodes[0]['cast']

['Tom Hanks',
 'Tim Allen',
 'Don Rickles',
 'Jim Varney',
 'Wallace Shawn',
 'John Ratzenberger',
 'Annie Potts',
 'John Morris',
 'Erik von Detten',
 'Laurie Metcalf',
 'R. Lee Ermey',
 'Sarah Freeman',
 'Penn Jillette']

In [6]:
for node in nodes:
    try:
        print(literal_eval(node['belongs_to_collection']))
    except:
        pass

{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}
{}
{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}
{}
{'id': 645, 'name': 'James Bond Collection', 'poster_path': '/HORpg5CSkmeQlAolx3bKMrKgfi.jpg', 'backdrop_path': '/6VcVl48kNKvdXOZfJPdarlUGOsk.jpg'}
{'id': 117693, 'name': 'Balto Collection', 'poster_path': '/w0ZgH6Lgxt2bQYnf1ss74UvYftm.jpg', 'backdrop_path': '/9VM5LiJV0bGb1st1KyHA3cVnO2G.jpg'}
{}
{}
{'id': 91698, 'name': 'Chili Palmer Collection', 'poster_path': '/ae3smJDdWrMJ77tDpYOrpo4frKq.jpg', 'backdrop_path': '/uWaANGQeoSs5vSP1CWtlkDrkqei.jpg'}
{}
{}
{}
{}
{}
{'id': 9435, 'name': 'Babe Collection', 'poster_path': '/jgnah0k4AumfcZE4HHMJywYHvG6.jpg', 'backdrop_path': '/zRWuGdIlkWqXMQcs1t3sPy1XQ6x.jpg'}
{}
{'id': 9818, 'name': 'Mortal Kombat Collection', 'poster_path': '/nauCll

In [7]:
from tqdm import tqdm

links = []

# add links between movies and genres

for i, genre in tqdm(enumerate(genres)):
    for node in nodes:
        if genre in node['genres']:
            link = {'source': i, 'target': node['id']}
            links.append(link)

# add links between movies and sieries


# add links between movies and actors

#for node in nodes:
#    for actor in node['cast'][:5]:
#            link = {'source': node['id'], 'target': actor}
#            links.append(link)

32it [00:00, 2205.61it/s]


In [1]:
for node in nodes:
    if node['belongs_to_collection'] != '':
        link = {'source': node['id'], 'target': node['belongs_to_collection']['name']}
        links.append(link)

NameError: name 'nodes' is not defined

In [9]:
nodes_output = [{'id': node['id'], 'title': node['title']} for node in nodes]
output = {'nodes': nodes_output, 'links': links}

import json
with open('data.json', 'w') as f:
    json.dump(output, f)